In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from beavermap import BeaverMap
import os 

In [10]:
bm = BeaverMap(
    h5_file = './data/al2o3_m330p0/al2o3_m330p0.h5',
    poni_file = './data/m330p0.poni',
    mask_file = './data/mask.edf',
    chunk_size = 100,
    location = '1.1/measurement/eiger',
    nworkers = 1
)

In [11]:
import numpy as np 
n_images = 7200

image_range = np.arange(7200)

chunks = bm.data_chunker(n_images,600)


In [12]:
#os.environ['OMP_NUM_THREADS'] = '1'
#max_sum = bm.max_sum()

In [13]:
#os.environ['OMP_NUM_THREADS'] = '10'
#max_sum = bm.max_sum()

In [14]:
#os.environ['OMP_NUM_THREADS'] = '1'
#bm.nworkers = 4 
#max_sum = bm.max_sum()

In [15]:
os.environ['OMP_NUM_THREADS'] = '1'
bm.nworkers = 4 
integration = bm.integrate(regions = [[0,3]],chunksize=600)

performing integration.0 600.0/600|100%|███████████████████████| 01:22<    00:00
performing integration.1 600.0/600|100%|███████████████████████| 01:24<    00:00
performing integration.2 600.0/600|100%|███████████████████████| 01:23<    00:00
performing integration.3 600.0/600|100%|███████████████████████| 01:27<    00:00
performing integration.4 600.0/600|100%|███████████████████████| 01:24<    00:00
performing integration.5 600.0/600|100%|███████████████████████| 01:37<    00:00
Process SpawnProcess-27:
Traceback (most recent call last):
performing integration.6 600.0/600|100%|███████████████████████| 01:29<    00:00
performing integration.7 600.0/600|100%|███████████████████████| 01:23<    00:00
performing integration.8 600.0/600|100%|███████████████████████| 01:23<    00:00
performing integration.9 600.0/600|100%|███████████████████████| 01:24<    00:00
performing integration.10 600.0/600|100%|██████████████████████| 01:22<    00:00
performing integration.11 600.0/600|100%|████████

In [16]:
integration

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       ...,


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]